# 행렬 성질로 구분

## 실수행렬(Real Matrix)
### 대칭(Symmetric)
- 양의정부호 행렬(positive definite)
     x^T *Ax>0
- 음의 정부호 행렬 (Negative definite)
- 부정부호 행렬(Indefinite)
### 반대칭(Non-symmetric)

## 복소수행렬(Complex Matrix)
### 에르미트(Hermitian) 
 - A=A* A=not A^T
 - 양의 정부호 행렬(positive definite)
     x*Ax>0
 - 음의 정부호 행렬
 - 부정부호 행렬
### Non-Hermitian
 - R(x*Ax)
 - 
### Complex symmetric matrix
- boundary integral equations
- block symmetric daigonal
## positive definite 풀이
 ### 콜레스키 분해(Cholesky decomposition)
 - A=R^TR=LL^T
 ### LDL 분해(LDL decomposition)
 - 삼각행렬
     - UDU^T=LDL^T
     - UDU*=LDL*
     
## 에르미트, 대칭행렬에서 definite 모를때 
 ### Daigonal pivoting methon
 - A=UDU^T=LDL^T (Block symmetric diagonal)
 - A=UDU*=LDL* (Block Hermitian diagonal)

# 행렬 모양으로 구분

## Band Matrix
 ### bandwidth
 - tridaigonal matrix
 
## 퇴플리츠 Toeplitz Matrix
 ### Levinson_Durbin recursion ~n^2
 
## 순환행렬 Circulant Matrix
 - 이미지 시각화, 신호, 암호학 등
 - discrete Furuier transform
 - fast Fourier transform ~nlog(n)

# 수치해석(Iterative Methods)
    - 야코비안,가우스요르단,SOR,BICG,GMRES

# 역행렬 구하는 것보다 행렬방정식으로 풀어야 하는 이유

## 역행렬로 해를 찾음
 - a^-1 구할때 노력 ~n^3
 - a^-1^b 구할때 노력 ~n^2
 
## Ax=b로 해를 찾음
 - 가우스 소거법 Ax=b를 푸는 노력 ~n^3
 - LU분해 ~n^3, LUx=b ~n^2
 
## 수치적 정확도가 올라감
 - 컴퓨터에서 수를 제한된 소수점으로 표현하여 역행렬을 이용하면 오차 발생 가능성
 
## 계산 속도가 올라감
 - 희소 행렬의 역행렬과 LU 분해를 했을 때 lu분해가 더 희소한 경우가 많음

# Scipy linalg

In [9]:
from scipy import linalg
import numpy as np

## 행렬식 구하기

### 기본알고리즘 : LU decomposition
 - Lapack 함수: zgetrf(complex128) dgetrf(float64)

In [12]:
A=np.array([[1,5,8],[2,3,4],[1,9,-2]])
det = linalg.det(A)
print(det)

118.0


## 역행렬 구하기
### 기본알고리즘: LU deconposition
- solve LUA^-1 =1 (backward phase)
- Lapack 함수: getrf(LU), getrf(inverse from triangluar matrix)

In [15]:
inv_a=linalg.inv(A)
print(inv_a)
# singular marix 에러(수치적)

[[-0.3559322   0.69491525 -0.03389831]
 [ 0.06779661 -0.08474576  0.10169492]
 [ 0.12711864 -0.03389831 -0.05932203]]


## Ax=b 풀기
### linalg.solve(A,b,assume_a="gen") 
 - gen(행렬 특성모름) ->LU 분해, gesv 함수
 - sym(대칭행렬)-> diagonal pivoting, sysv 함수(complex symmetric 가능) 에르미트X
 - her(에르미트 행렬) -> diagonal pivotiong(LDL*), hesv 함수
 - pos(양의 정부호 행렬) -> Cholesky decomposition, posv 함수
 - assume_a 는 행렬 특성 잘못넣어도 오류 표시 X

In [16]:
b = np.ones( (3,), dtype=np.float64 )

A_sing = np.array( [ [1,3,4], [-4,2,-6], [-3,-2,-7] ], 
         dtype=np.float64)

A_gen = np.array( [ [0,1,2], [1,0,3], [4,-3,8] ], 
        dtype=np.float64)

A_sym = np.array( [ [1,2,1], [2,1,3], [1,3,1] ], 
        dtype=np.float64)

A_sym_c = np.array( [ [1,2-1j,1+2j], [2-1j,1,3], [1+2j,3,1] ], 
          dtype=np.complex128)

A_her = np.array( [ [1,2+1j,1-2j], [2-1j,1,3], [1+2j,3,1] ], 
        dtype=np.complex128)

A_pos = np.array( [ [2,-1,0], [-1,2,-1], [0,-1,2] ], 
        dtype=np.float64)

In [21]:
x = linalg.solve(A_pos,b, assume_a="gen")

print(x)

[1.5 2.  1.5]


## 삼각행렬로 풀기
### 상삼각행렬
 - linalg.solve_triangular(A,b,lower=False)
 - trtrs 함수

In [23]:
A = np.array([[1,0,0,0],[1,4,0,0],[5,0,1,0],[8,1,-2,2]],
    dtype=np.float64)

b = np.array([1,2,3,4],dtype=np.float64)

x = linalg.solve_triangular(A,b,lower=True)

print(x)

[ 1.     0.25  -2.    -4.125]


## 정확한 수치일까? Allclose
### x= 수치적 계산으로 근사된 값
 - Ax-b 가 0에 가까운가 == Ax 와 b가 비슷한가

In [25]:
b = np.ones( (3,), dtype=np.float64 )

A = np.array( [ [2,-1,0], [-1,2,-1], [0,-1,2] ], 
        dtype=np.float64)

x = linalg.solve(A,b,assume_a="pos")

print(x)

zr = np.zeros((3,),dtype=np.float64)

bool_close = np.allclose( A@x-b, zr )
print(bool_close)

#  Ax와 b를 비교 가능
bool_close = np.allclose( A@x, b )
print(bool_close)

[1.5 2.  1.5]
True
True


In [49]:
a=linalg.hilbert(3)
inv_a=linalg.inv(a)
b = np.ones( (3,), dtype=np.float64 )

x1=inv_a@b
x2=linalg.solve(A,b,assume_a="gen")

print(x1)
print(x2)
bool_close = np.allclose( A@x-b, A@x2-b )
print(bool_close)

[  3. -24.  30.]
[1.5 2.  1.5]
True
